In [2]:
import pandas as pd
import pymongo
import mysql.connector
from bson import json_util

In [25]:
# this is simple ETL process

def ETL():
    extracted_data=Extract()
    if(len(extracted_data)>0):
       transfromed_data = Transform(extracted_data)
       if(len(transfromed_data)>0):
            Load(transfromed_data)
    
def Extract ():
    try:
        mongodb = pymongo.MongoClient("mongodb://localhost:27017/")
        database=mongodb['amazon']
        collection=database['Sales']
        datas=collection.find()
        datas=json_util.dumps(datas)
        datas= json_util.loads(datas)
        return datas
    except Exception as e:
        print("Error")
def Transform(data):
    data=Extract()
    updateddata=[]
    for i in range(0,len(data)):
        updated_data={}
        updated_data['product_name']=data[i]['product_name']
        updated_data['category']=data[i]['category']
        updated_data['rating']=data[i]['rating']
        updated_data['rating_count']=data[i]['rating_count']
        updated_data['actual_price']=data[i]['actual_price']
        updateddata.append(updated_data)
    # updateddata=json_util.loads(updateddata)
    return updateddata
def Load(data):
    mydb=mysql.connector.connect(
        host="localhost",
        user="root",
        password="Friday",
        database="ETL_process",
    )
    mycursor=mydb.cursor()
    # Insert data into database
    if(len(data)>0):
        for record in data:
            sql="INSERT INTO ETL_process (ProductName,Category,Rating,RatingCount,ActualPrice) VALUES (%s,%s,%s,%s,%s)"
            val=(record['product_name'],record['category'],record['rating'],record['rating_count'],record['actual_price'])
            # print(record['product_name'])
            mycursor.execute(sql, val)
        #Commit changes to database
        mydb.commit()
        # Close connection to database
        mydb.close()
        print("Data Loaded in WareHouse Successfully")
        
    else:
        print("Data is empty")
    # pass    
# Load()
ETL()


Data Loaded in WareHouse Successfully
